In [1]:
!pip freeze | grep boto3

aioboto3 @ file:///home/conda/feedstock_root/build_artifacts/aioboto3_1742196379442/work
boto3==1.38.18


In [32]:
import uuid
import os
from pathlib import Path

def get_or_create_unique_resources_identifier():
    unique_resources_identifier_file = '../.unique_resources_identifier'
    
    if os.path.exists(unique_resources_identifier_file):
        with open(unique_resources_identifier_file, 'r') as f:
            return f.read().strip()
    else:
        unique_resources_identifier = str(uuid.uuid4())[:8]
        with open(unique_resources_identifier_file, 'w') as f:
            f.write(unique_resources_identifier)
        return unique_resources_identifier

unique_resources_identifier = get_or_create_unique_resources_identifier()
resource_suffix = f"{unique_resources_identifier}"
print("Your resource suffix is", resource_suffix)

Your resource suffix is travel_booking_ca3e24de


In [33]:
import boto3
import os
import json
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0',
    'amazon.titan-embed-text-v2:0',
    'amazon.titan-embed-image-v1',
    'amazon.titan-text-express-v1',
    'amazon.titan-text-lite-v1',
    'ai21.j2-mid-v1',
    'ai21.j2-ultra-v1',
    'cohere.command-text-v14',
    'cohere.embed-english-v3',
    'cohere.embed-multilingual-v3',
    'meta.llama2-13b-chat-v1',
    'meta.llama2-70b-chat-v1',
    'amazon.nova-pro-v1:0'
]
print(account_id)
curr_month = datetime.now()

225584133776


In [4]:
# Define agent names and IDs
supervisor_agent_name = f"supervisor-agent-{resource_suffix}"
supervisor_lambda_name = f"fn-supervisor-agent-{resource_suffix}"

In [34]:
# Import helper modules
import sys
sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

In [35]:
# Retrieve stored agent information
%store -r hr_agent_alias_id
%store -r flight_agent_alias_id
%store -r hotel_agent_alias_id
%store -r hr_agent_alias_arn
%store -r flight_agent_alias_arn
%store -r hotel_agent_alias_arn

In [16]:
print('hr_agent_alias_id'+ hr_agent_alias_id)
print('flight_agent_alias_id'+ flight_agent_alias_id)
print('hotel_agent_alias_id'+ hotel_agent_alias_id)
print('hr_agent_alias_arn'+ hr_agent_alias_arn)
print('flight_agent_alias_arn'+ flight_agent_alias_arn)
print('hotel_agent_alias_arn'+ hotel_agent_alias_arn)

hr_agent_alias_idJUZ0EVGCPO
flight_agent_alias_id0JR7ILBRBW
hotel_agent_alias_id0TJYBOB5UV
hr_agent_alias_arnarn:aws:bedrock:us-west-2:225584133776:agent-alias/XQJ6KAV7P1/JUZ0EVGCPO
flight_agent_alias_arnarn:aws:bedrock:us-west-2:225584133776:agent-alias/GFBQWRL2CH/0JR7ILBRBW
hotel_agent_alias_arnarn:aws:bedrock:us-west-2:225584133776:agent-alias/SLJHZCNZNF/0TJYBOB5UV


In [36]:
# Create supervisor agent with detailed instructions
supervisor_agent_instructions = """
You are the Supervisor Agent responsible for managing the entire travel booking workflow for employees. You will coordinate between the HR Agent, Flight Booking Agent, and Hotel Booking Agent to provide a complete travel booking experience.

When a user starts a conversation:

1. First, identify the user by asking for their Employee ID.

2. Once you have the Employee ID, tell the user you're checking their profile with HR.
   - Mention: "Let me check your profile with our HR system."
   - Then ask about their travel needs: "What are your travel plans? Please provide your origin, destination, and dates."

3. When the user provides travel details:
   - Acknowledge their request: "Thank you for providing your travel details."
   - Tell them you'll check flight options: "Let me find suitable flight options for you."
   - After discussing flights, move to hotel options: "Now, let's find a hotel for your stay."

4. After collecting both flight and hotel information:
   - Summarize the bookings: "Here's a summary of your bookings:"
   - List flight details (airline, flight number, date, time, price)
   - List hotel details (name, check-in/out dates, room type, price)
   - Calculate and show total cost

5. Conclude with payment information:
   - "Your total cost is $X. Please proceed to our payment portal to complete your booking."

Important guidelines:
- Always maintain a professional and helpful tone
- Keep the conversation flowing naturally between different booking stages
- If you need specific employee information, mention that you're checking with HR
- Present flight and hotel options clearly with all relevant details
- Ensure all bookings comply with company travel policies
- If any issues arise, explain them clearly and suggest alternatives

Remember: You are the coordinator between specialized agents. Your role is to make the booking process seamless for the user while ensuring all company policies are followed.
"""



In [38]:
# Create the supervisor agent
supervisor_agent = agents.create_agent(
    agent_name=supervisor_agent_name,
    agent_description="Supervisor agent for orchestrating travel booking workflow",
    agent_instructions=supervisor_agent_instructions,
    model_ids=['anthropic.claude-3-sonnet-20240229-v1:0'],
    agent_collaboration='SUPERVISOR_ROUTER'
)


ConflictException: An error occurred (ConflictException) when calling the CreateAgent operation: Could not perform Create operation, since the supervisor-agent-travel_booking_ca3e24de (id: NPEA338HVM) with the same name supervisor-agent-travel_booking_ca3e24de already exists

In [39]:
supervisor_agent_id =supervisor_agent[0]
print(supervisor_agent_id)

NPEA338HVM


In [40]:
# Add sub-agent groups
hr_agent_group = {
    "sub_agent_association_name": "hr_agent",
    "sub_agent_instruction": "HR agent for employee information and policy enforcement",
    "sub_agent_alias_arn": hr_agent_alias_arn,
    'relay_conversation_history': 'TO_COLLABORATOR'

}

In [41]:
flight_agent_group = {
    "sub_agent_association_name": "flight_agent",
    "sub_agent_instruction": "Flight booking agent for searching and booking flights",
    "sub_agent_alias_arn": flight_agent_alias_arn,
    'relay_conversation_history': 'TO_COLLABORATOR'

}

In [42]:
hotel_agent_group = {
    "sub_agent_association_name": "hotel_agent",
    "sub_agent_instruction": "Hotel booking agent for searching and booking accommodations",
    "sub_agent_alias_arn": hotel_agent_alias_arn,
    'relay_conversation_history': 'TO_COLLABORATOR'

}

In [43]:
sub_agents_list = [hr_agent_group,flight_agent_group,hotel_agent_group]

In [44]:
# Add sub-agent groups to supervisor agent
agents.associate_sub_agents(
    supervisor_agent_id, sub_agents_list
)

Waiting for agent status to change. Current status PREPARING
Agent id NPEA338HVM current status: PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id NPEA338HVM current status: PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id NPEA338HVM current status: PREPARED


('TA4NDUBC8N',
 'arn:aws:bedrock:us-west-2:225584133776:agent-alias/NPEA338HVM/TA4NDUBC8N')

In [45]:
# Create agent alias for multi-agent collaboration
supervisor_agent_alias_id, supervisor_agent_alias_arn = agents.create_agent_alias(
    supervisor_agent[0], 'v1'
)


In [47]:

%store supervisor_agent_alias_arn
%store supervisor_agent_alias_id
%store supervisor_agent_name
%store supervisor_agent_id


Stored 'supervisor_agent_alias_arn' (str)
Stored 'supervisor_agent_alias_id' (str)
Stored 'supervisor_agent_name' (str)
Stored 'supervisor_agent_id' (str)


In [48]:
# Test the supervisor agent
response = agents.invoke(
    """Hi, I'm employee E001 and I need to book a business trip to NYC next week. I need both flight and hotel.""", 
    supervisor_agent_id, enable_trace=True
)
print("====================")
print(response)



invokeAgent API request ID: 97af6b5d-7af7-46bd-aa0b-4ddba26a14aa
invokeAgent API session ID: dcb41804-34cc-11f0-b805-06efa4e78d78
---- Step 1 ----
Classifying request to immediately route to one collaborator if possible.
Routing classifier did not find a matching collaborator. Reverting to 'SUPERVISOR' mode.
Routing classifier took 0.8s, using 345 tokens (in: 332, out: 13).

---- Step 2 ----
Took 9.0s, using 1518 tokens (in: 1348, out: 170) to complete prior action, observe, orchestrate.
To start the travel booking process, I need to:
1. Verify the employee ID with the HR agent
2. Get the travel details from the user (origin, destination, dates)
3. Check flight options with the flight agent
4. Check hotel options with the hotel agent
5. Provide a summary of the bookings and total cost
6. Instruct the user on payment
Using sub-agent collaborator: 'hr_agent [XQJ6KAV7P1/JUZ0EVGCPO]' passing input text:
Verifying employee ID E001

You haven't provided agents names. To do so provide a dicti